
<img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Find EC2 Instances Scheduled to Retire soon using unSkript legos.</b>
</div>

<br>

<center><h2>Find EC2 Instances Scheduled to Retire</h2></center>

# Steps Overview
    1) Filter all AWS EC2 instances
    2) Get the event.code for the scheduled event instance-retirement.

Here we will use unSkript Filter ALL AWS EC2 Instances Lego. This lego takes region as input. This input is used to discover all the EC2 instances.

In [2]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_filter_ec2_instances_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})


@beartype
def aws_filter_ec2_instances(handle, region: str) -> List:
    """aws_filter_ec2_by_tags Returns an array of instances.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :rtype: Array of instances.
    """
    # Input param validation.

    ec2Client = handle.client('ec2', region_name=region)
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")

    result = []
    for reservation in res:
        for instance in reservation['Instances']:
            result.append(instance['InstanceId'])
    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')
task.configure(outputName="instance_ids")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_ec2_instances, lego_printer=aws_filter_ec2_instances_printer, hdl=hdl, args=args)

Here we will use unSkript Get AWS Scheduled to Retire Instances Lego. This lego takes instance_ids list and region as input. This input is used to discover EC2 instances which are schedule to retire soon.

In [7]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_get_schedule_to_retire_instances(
    handle,
    instance_ids: list,
    region: str,
) -> List:

    ec2client = handle.client('ec2', region_name=region)
    instances = []
    response = ec2client.describe_instance_status(
        Filters=[
        {
            'Name': 'event.code',
            'Values': ['instance-retirement']}],
        InstanceIds=instance_ids)

    for instance in response['InstanceStatuses']:
        instance_id = instance['InstanceId']
        instances.append(instance_id)

    return instances


def aws_get_schedule_to_retire_instances_printer(output):
    if output is None:
        return
    pprint.pprint({"Instances": output})

task = Task(Workflow())
task.configure(inputParamsJson='''{
    "instance_ids": "instance_ids",
    "region": "Region"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_schedule_to_retire_instances, lego_printer=aws_get_schedule_to_retire_instances_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to perform AWS this runbook Get the EC2 instances which are schedule to retire soon. To view the full platform capabilities of unSkript please visit https://unskript.com